# Multi-Symbol Wheel Strategy Backtest v2

Enhanced backtest with realistic market simulation.

**New Features in v2:**
- **Transaction costs**: IBKR commissions and exchange fees
- **Slippage modeling**: Bid-ask spread based execution simulation
- **Liquidity filters**: Reject illiquid options (wide spreads, low size)
- **Exit fallback cascade**: Daily close → expiry logic when minute data unavailable
- **Fixed cost basis**: Proper cash-secured/margin calculation
- **Gross vs Net P&L**: Track impact of costs and slippage

**Existing Features:**
- Multi-symbol support (TSLA, AAPL, etc.)
- Caching of API responses
- Technical filter (Bollinger Bands/SMA)
- Configurable DTE (trading vs calendar days)

## 1. Imports & Setup

In [58]:
from pathlib import Path
from dotenv import load_dotenv
import os
import sys

# Load environment variables
env_path = Path("/Users/samuelminer/Projects/nissan_options/wheel_strategy/.env")
load_dotenv(env_path, override=True)
assert os.getenv("DATABENTO_API_KEY"), "DATABENTO_API_KEY not found"

import numpy as np
import pandas as pd
import databento as db
import pandas_market_calendars as mcal
from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_vollib.black_scholes.greeks.analytical import delta

# Initialize clients
client = db.Historical()
nyse = mcal.get_calendar("NYSE")

print("Setup complete")

Setup complete


## 3b. Intermediate Step Caching

Cache dataframes at each processing step for faster debugging and inspection.

In [59]:
def get_step_cache_path(step_name, ticker, date):
    """
    Get cache path for intermediate processing step.
    
    Args:
        step_name: Name of processing step (e.g., 'parsed_options', 'candidates', etc.)
        ticker: Ticker symbol
        date: Date string (YYYY-MM-DD)
    
    Returns:
        Full path to cache file
    """
    cache_name = f"step_{step_name}_{ticker}_{date}"
    return os.path.join(CONFIG['cache_dir'], f"{cache_name}.parquet")


def load_step_cache(step_name, ticker, date):
    """
    Load cached dataframe for a processing step.
    
    Returns:
        DataFrame if cached, None if not found
    """
    path = get_step_cache_path(step_name, ticker, date)
    if os.path.exists(path):
        print(f"  [CACHE HIT] Loading step: {step_name} for {ticker} on {date}")
        return pd.read_parquet(path)
    return None


def save_step_cache(df, step_name, ticker, date):
    """
    Save dataframe to cache for a processing step.
    """
    path = get_step_cache_path(step_name, ticker, date)
    df.to_parquet(path)
    print(f"  [CACHE SAVE] Saved step: {step_name} for {ticker} on {date}")


def clear_step_cache_for_date(ticker, date):
    """
    Clear all step caches for a specific ticker/date combination.
    Useful when you want to reprocess a specific date.
    """
    pattern = f"step_*_{ticker}_{date}.parquet"
    cache_dir = CONFIG['cache_dir']
    
    import glob
    files = glob.glob(os.path.join(cache_dir, pattern))
    
    for f in files:
        os.remove(f)
        print(f"  [CACHE CLEAR] Removed {os.path.basename(f)}")
    
    if len(files) == 0:
        print(f"  [CACHE CLEAR] No step caches found for {ticker} on {date}")


# Example usage in process_entry_date():
# 
# # After parsing symbols
# df_parsed = load_step_cache('parsed_options', ticker, entry_date)
# if df_parsed is None:
#     df_parsed = parse_option_symbols(df_opts)
#     save_step_cache(df_parsed, 'parsed_options', ticker, entry_date)
#
# # After filtering candidates
# candidates = load_step_cache('candidates', ticker, entry_date)
# if candidates is None:
#     candidates = chain_snapshot[...filtering logic...]
#     save_step_cache(candidates, 'candidates', ticker, entry_date)

print("Intermediate step caching functions defined")
print("")
print("Available steps to cache:")
print("  - 'parsed_options': After parse_option_symbols()")
print("  - 'chain_with_dte': After add_trading_dte()")
print("  - 'chain_with_greeks': After IV/delta calculation")
print("  - 'candidates': After initial filters (DTE, delta, type)")
print("  - 'liquidity_filtered': After liquidity filters")
print("  - 'backtest_candidates': Final candidates ready for exit strategy")
print("  - 'exits': Exit strategy results")
print("")
print("To clear caches: clear_step_cache_for_date('TSLA', '2023-06-06')")

Intermediate step caching functions defined

Available steps to cache:
  - 'parsed_options': After parse_option_symbols()
  - 'chain_with_dte': After add_trading_dte()
  - 'chain_with_greeks': After IV/delta calculation
  - 'candidates': After initial filters (DTE, delta, type)
  - 'liquidity_filtered': After liquidity filters
  - 'backtest_candidates': Final candidates ready for exit strategy
  - 'exits': Exit strategy results

To clear caches: clear_step_cache_for_date('TSLA', '2023-06-06')


## 3c. Example: Using Step Caching

The `process_entry_date` function now caches intermediate steps. Here's how it works:

**Cached Steps:**
1. `parsed_options` - After parsing OPRA symbols
2. `chain_with_dte` - After calculating DTE
3. `chain_with_greeks` - After IV/delta calculation
4. `candidates` - After DTE/delta filters
5. `liquidity_filtered` - After liquidity filters
6. `backtest_candidates` - Final candidates ready for backtest
7. `exits` - Exit strategy results

**Cache Benefits:**
- Speed up re-runs when testing different parameters
- Inspect intermediate results for debugging  
- Skip expensive calculations (IV, delta) on repeat runs

**Example: Inspect a Specific Step**
```python
# Load cached candidates for TSLA on 2023-06-06
candidates = load_step_cache('candidates', 'TSLA', '2023-06-06')
if candidates is not None:
    print(f"Found {len(candidates)} candidates")
    print(candidates[['symbol', 'strike', 'dte', 'delta']].head())
```

**Clear Cache for Specific Date:**
```python
# Clear all step caches for TSLA on 2023-06-06
clear_step_cache_for_date('TSLA', '2023-06-06')
```

In [60]:
# =============================================================================
# EXAMPLE: Inspect cached intermediate steps
# =============================================================================

# Uncomment to inspect a specific step for a ticker/date:

# Step 1: Load parsed options
# df = load_step_cache('parsed_options', 'TSLA', '2023-06-06')
# if df is not None:
#     print(f"Parsed options: {len(df)} rows")
#     print(df[['symbol', 'expiration', 'strike', 'call_put']].head())

# Step 2: Load chain with DTE
# df = load_step_cache('chain_with_dte', 'TSLA', '2023-06-06')
# if df is not None:
#     print(f"Chain with DTE: {len(df)} rows")
#     print(df[['symbol', 'dte', 'expiration']].head())

# Step 3: Load chain with greeks
# df = load_step_cache('chain_with_greeks', 'TSLA', '2023-06-06')
# if df is not None:
#     print(f"Chain with greeks: {len(df)} rows")
#     print(df[['symbol', 'dte', 'iv', 'delta', 'mid']].head())

# Step 4: Load filtered candidates
# df = load_step_cache('candidates', 'TSLA', '2023-06-06')
# if df is not None:
#     print(f"Candidates after filters: {len(df)} rows")
#     print(df[['symbol', 'strike', 'dte', 'delta', 'mid']].head())

print("Step caching examples ready (uncomment to use)")

Step caching examples ready (uncomment to use)


## 2. Configuration

In [61]:
CONFIG = {
    # ==========================================================================
    # TICKERS & DATES
    # ==========================================================================
    'tickers': ['TSLA', 'AAPL'],
    'entry_dates': ['2023-06-06'],  # Or use 'start_date' and 'end_date' below
    # 'start_date': '2023-06-01',
    # 'end_date': '2023-06-30',

    # ==========================================================================
    # GENERAL SETTINGS
    # ==========================================================================
    'timezone': 'America/New_York',
    'cache_dir': '../cache/',
    'risk_free_rate': 0.04,

    # ==========================================================================
    # OPTION FILTERS
    # ==========================================================================
    'min_dte': 5,
    'max_dte': 10,
    'min_delta': 0.00,
    'max_delta': 0.15,
    'option_type': 'P',  # 'P' for puts, 'C' for calls

    # ==========================================================================
    # DTE CALCULATION
    # ==========================================================================
    'use_trading_days_for_dte_filter': True,   # True = trading days, False = calendar days
    'use_trading_days_for_exit': True,         # True = trading days, False = calendar days

    # ==========================================================================
    # EXIT STRATEGY
    # ==========================================================================
    'profit_target_pct': 0.50,  # Exit at 50% of premium
    'exit_dte': 0,              # Days before expiration to force exit (0 = expiration)

    # ==========================================================================
    # MARGIN SETTINGS
    # ==========================================================================
    'margin': {
        'use_margin': False,        # False = cash-secured (full collateral)
        'margin_requirement': 0.20, # 20% margin if use_margin=True
    },

    # ==========================================================================
    # TRANSACTION COSTS (IBKR)
    # ==========================================================================
    'costs': {
        'enabled': True,
        'commission_per_contract': 0.25,    # IBKR tiered pricing
        'exchange_fees_per_contract': 0.05, # OCC, exchange fees
        'assignment_fee': 0.00,             # IBKR doesn't charge for assignment
    },

    # ==========================================================================
    # SLIPPAGE
    # ==========================================================================
    'slippage': {
        'enabled': True,
        'model': 'spread_percentage',  # 'none', 'spread_percentage', 'fixed'
        'spread_pct': 0.25,            # Lose 25% of half-spread on each trade
        'fixed_amount': 0.02,          # Fixed $ per share if model='fixed'
    },

    # ==========================================================================
    # LIQUIDITY FILTERS
    # ==========================================================================
    'liquidity': {
        'enabled': True,
        'max_spread_pct': 0.15,  # Max 15% bid-ask spread relative to mid
        'min_bid_size': 10,      # Minimum contracts on bid
        'min_ask_size': 10,      # Minimum contracts on ask
    },

    # ==========================================================================
    # EXIT FALLBACK (when minute data unavailable)
    # ==========================================================================
    'exit_fallback': {
        'use_daily_close': True,   # Fallback 1: Use daily OHLCV close
        'use_expiry_logic': True,  # Fallback 2: ITM=intrinsic value, OTM=$0
    },

    # ==========================================================================
    # TECHNICAL FILTER (Bollinger Bands)
    # ==========================================================================
    'technical_filter_enabled': False,
    'bb_window': 20,
    'bb_std': 2.0,
    'require_sma_entry': True,
    'require_bb_entry': False,
}

# =============================================================================
# Initialize
# =============================================================================
os.makedirs(CONFIG['cache_dir'], exist_ok=True)

# Convert date range to trading days if specified
if 'start_date' in CONFIG and 'end_date' in CONFIG:
    start = pd.Timestamp(CONFIG['start_date'])
    end = pd.Timestamp(CONFIG['end_date'])
    trading_days = nyse.valid_days(start_date=start, end_date=end)
    CONFIG['entry_dates'] = [d.strftime('%Y-%m-%d') for d in trading_days]
    print(f"Date range: {CONFIG['start_date']} to {CONFIG['end_date']}")
    print(f"Generated {len(CONFIG['entry_dates'])} trading days")
else:
    print(f"Entry dates: {CONFIG['entry_dates']}")

# Print configuration summary
print(f"\nTickers: {CONFIG['tickers']}")
print(f"Total combinations: {len(CONFIG['tickers'])} x {len(CONFIG['entry_dates'])} = {len(CONFIG['tickers']) * len(CONFIG['entry_dates'])}")
print(f"\n--- Feature Toggles ---")
print(f"Transaction costs: {'ON' if CONFIG['costs']['enabled'] else 'OFF'}")
print(f"Slippage: {'ON' if CONFIG['slippage']['enabled'] else 'OFF'} ({CONFIG['slippage']['model']})")
print(f"Liquidity filters: {'ON' if CONFIG['liquidity']['enabled'] else 'OFF'}")
print(f"Exit fallback: daily_close={CONFIG['exit_fallback']['use_daily_close']}, expiry_logic={CONFIG['exit_fallback']['use_expiry_logic']}")
print(f"Margin mode: {'MARGIN' if CONFIG['margin']['use_margin'] else 'CASH-SECURED'}")

Entry dates: ['2023-06-06']

Tickers: ['TSLA', 'AAPL']
Total combinations: 2 x 1 = 2

--- Feature Toggles ---
Transaction costs: ON
Slippage: ON (spread_percentage)
Liquidity filters: ON
Exit fallback: daily_close=True, expiry_logic=True
Margin mode: CASH-SECURED


## 3. Caching Functions

In [62]:
def get_cache_path(name):
    """Get full path for a cache file"""
    return os.path.join(CONFIG['cache_dir'], f"{name}.parquet")

def load_from_cache(name):
    """Load DataFrame from cache if it exists"""
    path = get_cache_path(name)
    if os.path.exists(path):
        print(f"  [CACHE HIT] Loading {name}")
        return pd.read_parquet(path)
    return None

def save_to_cache(df, name):
    """Save DataFrame to cache"""
    path = get_cache_path(name)
    df.to_parquet(path)
    print(f"  [CACHE SAVE] Saved {name}")

print("Caching functions defined")

Caching functions defined


## 3b. Cost, Slippage & Liquidity Functions

In [63]:
# =============================================================================
# COST BASIS CALCULATION
# =============================================================================
def calculate_cost_basis(strike, config):
    """
    Calculate cost basis (collateral required) for a cash-secured put.
    
    Args:
        strike: Strike price of the put
        config: CONFIG dict with margin settings
    
    Returns:
        Cost basis per contract (strike * 100 or margin-adjusted)
    """
    if config['margin']['use_margin']:
        return strike * 100 * config['margin']['margin_requirement']
    return strike * 100  # Full cash-secured


# =============================================================================
# TRANSACTION COSTS
# =============================================================================
def calculate_entry_costs(num_contracts, config):
    """
    Calculate total costs for entering a position.
    
    Returns:
        Total cost in dollars
    """
    if not config['costs']['enabled']:
        return 0.0
    
    commission = num_contracts * config['costs']['commission_per_contract']
    exchange_fees = num_contracts * config['costs']['exchange_fees_per_contract']
    return commission + exchange_fees


def calculate_exit_costs(num_contracts, exit_reason, config):
    """
    Calculate total costs for exiting a position.
    
    Args:
        num_contracts: Number of contracts
        exit_reason: 'profit_target', 'time_limit', 'expiry_itm', 'expiry_otm'
        config: CONFIG dict
    
    Returns:
        Total cost in dollars
    """
    if not config['costs']['enabled']:
        return 0.0
    
    # If assigned (ITM at expiry), may have assignment fee
    if exit_reason == 'expiry_itm':
        assignment = num_contracts * config['costs']['assignment_fee']
        return assignment  # No commission when assigned
    
    # If expired worthless (OTM), no costs
    if exit_reason == 'expiry_otm':
        return 0.0
    
    # Regular exit (buy to close)
    commission = num_contracts * config['costs']['commission_per_contract']
    exchange_fees = num_contracts * config['costs']['exchange_fees_per_contract']
    return commission + exchange_fees


# =============================================================================
# SLIPPAGE
# =============================================================================
def apply_entry_slippage(mid, bid, ask, config):
    """
    Apply slippage for SELLING (entry). Fill will be closer to bid.
    
    Args:
        mid: Mid price
        bid: Bid price
        ask: Ask price
        config: CONFIG dict with slippage settings
    
    Returns:
        Adjusted fill price (lower than mid when selling)
    """
    if not config['slippage']['enabled']:
        return mid
    
    model = config['slippage']['model']
    
    if model == 'none':
        return mid
    elif model == 'spread_percentage':
        # Lose a percentage of the half-spread
        half_spread = (ask - bid) / 2
        slippage = half_spread * config['slippage']['spread_pct']
        return mid - slippage  # Sell lower than mid
    elif model == 'fixed':
        return mid - config['slippage']['fixed_amount']
    else:
        return mid


def apply_exit_slippage(mid, bid, ask, config):
    """
    Apply slippage for BUYING (exit). Fill will be closer to ask.
    
    Args:
        mid: Mid price
        bid: Bid price  
        ask: Ask price
        config: CONFIG dict with slippage settings
    
    Returns:
        Adjusted fill price (higher than mid when buying)
    """
    if not config['slippage']['enabled']:
        return mid
    
    model = config['slippage']['model']
    
    if model == 'none':
        return mid
    elif model == 'spread_percentage':
        half_spread = (ask - bid) / 2
        slippage = half_spread * config['slippage']['spread_pct']
        return mid + slippage  # Buy higher than mid
    elif model == 'fixed':
        return mid + config['slippage']['fixed_amount']
    else:
        return mid


# =============================================================================
# LIQUIDITY FILTERS
# =============================================================================
def calculate_spread_pct(bid, ask):
    """Calculate bid-ask spread as percentage of mid price."""
    if bid <= 0 or ask <= 0:
        return float('inf')
    mid = (bid + ask) / 2
    if mid <= 0:
        return float('inf')
    return (ask - bid) / mid


def passes_liquidity_filter(row, config):
    """
    Check if an option passes liquidity filters.
    
    Args:
        row: DataFrame row with bid_px_00, ask_px_00, bid_sz_00, ask_sz_00
        config: CONFIG dict with liquidity settings
    
    Returns:
        (passes: bool, reason: str or None)
    """
    if not config['liquidity']['enabled']:
        return True, None
    
    bid = row.get('bid_px_00', 0)
    ask = row.get('ask_px_00', 0)
    bid_size = row.get('bid_sz_00', 0)
    ask_size = row.get('ask_sz_00', 0)
    
    # Check spread
    spread_pct = calculate_spread_pct(bid, ask)
    if spread_pct > config['liquidity']['max_spread_pct']:
        return False, f"spread_too_wide ({spread_pct:.1%} > {config['liquidity']['max_spread_pct']:.1%})"
    
    # Check bid size
    if bid_size < config['liquidity']['min_bid_size']:
        return False, f"bid_size_low ({bid_size} < {config['liquidity']['min_bid_size']})"
    
    # Check ask size
    if ask_size < config['liquidity']['min_ask_size']:
        return False, f"ask_size_low ({ask_size} < {config['liquidity']['min_ask_size']})"
    
    return True, None


print("Cost, slippage & liquidity functions defined")

Cost, slippage & liquidity functions defined


## 4. Helper Functions

In [64]:
def parse_option_symbols(df):
    """Parse OPRA symbols into components"""
    sym = df["symbol"]
    
    # Split ROOT and OPRA code
    root_and_code = sym.str.split(expand=True)
    df["root"] = root_and_code[0]
    code = root_and_code[1]
    
    # Expiration: YYMMDD
    df["expiration"] = pd.to_datetime(code.str[:6], format="%y%m%d")
    
    # Call/Put flag
    df["call_put"] = code.str[6]
    
    # Strike: in 1/1000 dollars
    strike_int = code.str[7:].astype("int32")
    df["strike"] = strike_int / 1000.0
    
    return df


def add_trading_dte(df, tz="America/New_York", use_trading_days=True):
    """
    Add days-to-expiration using NYSE calendar or calendar days.
    
    Args:
        df: DataFrame with ts_event and expiration columns
        tz: Timezone for event dates
        use_trading_days: If True, count trading days. If False, count calendar days.
    """
    out = df.copy()
    
    # Event dates from ts_event column
    event_dt = pd.to_datetime(out["ts_event"]).dt.tz_convert(tz).dt.normalize()
    event_days = pd.to_datetime(event_dt.dt.date)  # tz-naive
    
    # Expiration dates
    exp_dt = pd.to_datetime(out["expiration"])
    exp_days = pd.to_datetime(exp_dt.dt.date)  # tz-naive
    
    if use_trading_days:
        # Build trading calendar
        start_date = event_days.min().date()
        end_date = exp_days.max().date()
        
        schedule = nyse.valid_days(start_date=start_date, end_date=end_date)
        schedule = pd.to_datetime(schedule).normalize().tz_localize(None)
        
        cal_index = pd.Series(np.arange(len(schedule), dtype=np.int32), index=schedule)
        
        event_idx = cal_index.reindex(event_days).to_numpy()
        exp_idx = cal_index.reindex(exp_days).to_numpy()
        
        out["dte"] = (exp_idx - event_idx - 1).astype(np.int16)
    else:
        # Calendar days (simple subtraction)
        out["dte"] = (exp_days - event_days).dt.days.astype(np.int16)
    
    return out


def calculate_exit_dte_dates(expirations, exit_dte, use_trading_days=True):
    """
    Calculate dates at exit_dte days before expiration.
    
    Args:
        expirations: Series of expiration dates
        exit_dte: Number of days before expiration to exit (0 = expiration day)
        use_trading_days: If True, count trading days. If False, count calendar days
                          (will adjust to prior market day if lands on non-trading day)
    
    Returns:
        Series of exit dates (always valid trading days)
    """
    if exit_dte == 0:
        # Exit at expiration - return expiration dates (already trading days for options)
        return expirations.apply(lambda x: pd.Timestamp(x).normalize())
    
    min_exp = expirations.min()
    max_exp = expirations.max()
    
    start_date = min_exp - pd.Timedelta(days=60)
    end_date = max_exp
    
    schedule = nyse.schedule(start_date=start_date, end_date=end_date)
    trading_days = schedule.index.tz_localize(None)
    trading_days_set = set(trading_days)
    
    results = []
    for exp in expirations:
        exp_dt = pd.Timestamp(exp).normalize()
        
        if use_trading_days:
            # Count back trading days
            valid_days = trading_days[trading_days <= exp_dt]
            
            if len(valid_days) >= exit_dte:
                target_date = valid_days[-(exit_dte + 1)]  # +1 because expiration day is index -1
            else:
                target_date = valid_days[0] if len(valid_days) > 0 else exp_dt
        else:
            # Count back calendar days
            target_date = exp_dt - pd.Timedelta(days=exit_dte)
            
            # If target_date is not a trading day, find the prior trading day
            if target_date not in trading_days_set:
                prior_days = trading_days[trading_days < target_date]
                if len(prior_days) > 0:
                    target_date = prior_days[-1]
                else:
                    # Fallback to expiration if no prior trading days
                    target_date = exp_dt
        
        results.append(target_date)
    
    return pd.Series(results, index=expirations.index)


def compute_iv(row, r):
    """Compute implied volatility"""
    price = row["mid"]
    S = row["underlying_last"]
    K = row["strike"]
    t = row["dte"] / 365.0
    flag = "p" if row["call_put"] == "P" else "c"

    if not (np.isfinite(price) and np.isfinite(S) and np.isfinite(K) and t > 0):
        return np.nan
    if price <= 0 or S <= 0 or K <= 0:
        return np.nan

    try:
        return implied_volatility(price, S, K, t, r, flag)
    except Exception:
        return np.nan


def compute_delta(row, r):
    """Compute delta using IV"""
    sigma = row["iv"]
    if not np.isfinite(sigma):
        return np.nan

    S = row["underlying_last"]
    K = row["strike"]
    t = row["dte"] / 365.0
    flag = "p" if row["call_put"] == "P" else "c"

    return delta(flag, S, K, t, r, sigma)


print("Helper functions defined")

Helper functions defined


## 5. Data Fetch Functions

In [65]:
def fetch_options_snapshot(ticker, date):
    """Fetch option chain at 15:45 ET, with caching"""
    cache_name = f"options_{ticker}_{date}"
    
    # Try cache first
    cached = load_from_cache(cache_name)
    if cached is not None:
        return cached
    
    # Fetch from API
    print(f"  [API] Fetching options for {ticker} on {date}...")
    
    tz = CONFIG['timezone']
    start_time = pd.Timestamp(f"{date} 15:45", tz=tz)
    end_time = start_time + pd.Timedelta(minutes=1)
    
    data = client.timeseries.get_range(
        dataset='OPRA.PILLAR',
        schema='cmbp-1',
        symbols=f'{ticker}.OPT',
        stype_in='parent',
        start=start_time,
        end=end_time,
    )
    
    df = data.to_df(tz=tz).sort_values("ts_event")
    print(f"  [API] Fetched {len(df)} records")
    
    # Save to cache
    save_to_cache(df, cache_name)
    
    return df


def fetch_equity_price(ticker, date):
    """Fetch underlying price at 15:45 ET, with caching"""
    cache_name = f"equity_{ticker}_{date}"
    
    # Try cache first
    cached = load_from_cache(cache_name)
    if cached is not None:
        return cached['close'].iloc[0]
    
    # Fetch from API
    print(f"  [API] Fetching equity price for {ticker} on {date}...")
    
    tz = CONFIG['timezone']
    start_time = pd.Timestamp(f"{date} 15:45", tz=tz)
    end_time = start_time + pd.Timedelta(minutes=1)
    
    data = client.timeseries.get_range(
        dataset='XNAS.ITCH',
        symbols=[ticker],
        schema='ohlcv-1m',
        start=start_time,
        end=end_time,
        stype_in='raw_symbol'
    )
    
    df = data.to_df()
    print(f"  [API] Fetched equity price: ${df['close'].iloc[0]:.2f}")
    
    # Save to cache
    save_to_cache(df, cache_name)
    
    return df['close'].iloc[0]


def fetch_option_daily_ohlcv(symbol, start_date, end_date):
    """Fetch daily OHLCV for an option symbol, with caching"""
    # Clean symbol for cache filename
    cache_name = f"daily_{symbol.replace(' ', '_')}_{start_date}_{end_date}"
    
    # Try cache first
    cached = load_from_cache(cache_name)
    if cached is not None:
        return cached
    
    # Fetch from API
    print(f"  [API] Fetching daily OHLCV for {symbol} from {start_date} to {end_date}...")
    
    data = client.timeseries.get_range(
        dataset='OPRA.PILLAR',
        schema='ohlcv-1d',
        symbols=symbol,
        stype_in='raw_symbol',
        start=start_date,
        end=end_date,
    )
    
    df = data.to_df(tz=CONFIG['timezone'])
    print(f"  [API] Fetched {len(df)} daily records")
    
    # Save to cache
    save_to_cache(df, cache_name)
    
    return df


def fetch_option_1545_price(symbol, date):
    """Fetch option price at 15:45 ET for a specific date, with caching"""
    # Clean symbol for cache filename
    cache_name = f"option_1545_{symbol.replace(' ', '_')}_{date}"
    
    # Try cache first
    cached = load_from_cache(cache_name)
    if cached is not None:
        return cached['close'].iloc[0]
    
    # Fetch from API
    print(f"  [API] Fetching 15:45 price for {symbol} on {date}...")
    
    exit_time = pd.Timestamp(date).tz_localize(CONFIG['timezone']).replace(hour=15, minute=45)
    
    data = client.timeseries.get_range(
        dataset='OPRA.PILLAR',
        schema='ohlcv-1m',
        symbols=symbol,
        stype_in='raw_symbol',
        start=exit_time,
        end=exit_time + pd.Timedelta(minutes=1),
    )
    
    df = data.to_df(tz=CONFIG['timezone'])
    
    if len(df) > 0:
        exit_price = df.iloc[0]['close']
        print(f"  [API] Fetched price: ${exit_price:.2f}")
        
        # Save to cache
        save_to_cache(df, cache_name)
        
        return exit_price
    else:
        print(f"  [API] No data available")
        return None


print("Data fetch functions defined")

Data fetch functions defined


## 5b. Exit Price Fallback Function

In [66]:
def get_exit_price(symbol, exit_date, expiration, strike, underlying_price, config):
    """
    Get exit price with fallback cascade when minute data unavailable.
    
    Fallback order:
    1. Try 15:45 minute-level data
    2. Try daily OHLCV close for exit_date
    3. Use expiry logic (ITM = intrinsic value, OTM = $0)
    
    Args:
        symbol: Option symbol (e.g., 'TSLA  230721P00200000')
        exit_date: Target exit date (pd.Timestamp)
        expiration: Option expiration date (pd.Timestamp)
        strike: Strike price
        underlying_price: Current underlying price (for ITM/OTM check)
        config: CONFIG dict with exit_fallback settings
    
    Returns:
        (price: float or None, method: str)
        method is one of: 'minute_data', 'daily_close', 'expiry_itm', 'expiry_otm', 'no_data'
    """
    exit_date_normalized = pd.Timestamp(exit_date).tz_localize(None).normalize()
    expiration_normalized = pd.Timestamp(expiration).tz_localize(None).normalize()
    
    # 1. Try 15:45 minute-level data
    try:
        price = fetch_option_1545_price(symbol, exit_date_normalized.date())
        if price is not None:
            return price, 'minute_data'
    except Exception as e:
        print(f"    [FALLBACK] Minute data fetch failed: {e}")
    
    # 2. Try daily OHLCV close
    if config['exit_fallback']['use_daily_close']:
        try:
            # Fetch just that one day
            df_daily = fetch_option_daily_ohlcv(
                symbol, 
                exit_date_normalized, 
                exit_date_normalized + pd.Timedelta(days=1)
            )
            if len(df_daily) > 0:
                close_price = df_daily.iloc[-1]['close']
                print(f"    [FALLBACK] Using daily close: ${close_price:.2f}")
                return close_price, 'daily_close'
        except Exception as e:
            print(f"    [FALLBACK] Daily close fetch failed: {e}")
    
    # 3. Use expiry logic (only if exit_date >= expiration)
    if config['exit_fallback']['use_expiry_logic'] and exit_date_normalized >= expiration_normalized:
        # For a PUT: ITM when underlying < strike
        if underlying_price < strike:
            intrinsic = strike - underlying_price
            print(f"    [FALLBACK] Expiry ITM - intrinsic value: ${intrinsic:.2f}")
            return intrinsic, 'expiry_itm'
        else:
            print(f"    [FALLBACK] Expiry OTM - worthless")
            return 0.0, 'expiry_otm'
    
    # No data available
    print(f"    [FALLBACK] No exit price available")
    return None, 'no_data'


def fetch_underlying_price_for_date(ticker, date):
    """
    Fetch underlying price for a specific date (for expiry ITM/OTM calculation).
    Uses cached equity data or fetches new.
    """
    try:
        return fetch_equity_price(ticker, date)
    except Exception:
        # If we can't get the exact date, try to get most recent
        try:
            df = fetch_equity_history(ticker, date, lookback_days=5)
            if len(df) > 0:
                return df.iloc[-1]['close']
        except Exception:
            pass
    return None


print("Exit fallback functions defined")

Exit fallback functions defined


## 5b. Technical Filter Functions

In [67]:
def fetch_equity_history(ticker, end_date, lookback_days=60):
    """Fetch daily equity OHLCV data for technical analysis, with caching"""
    # Calculate start date with buffer for lookback
    end_dt = pd.Timestamp(end_date)
    start_dt = end_dt - pd.Timedelta(days=lookback_days)

    cache_name = f"equity_daily_{ticker}_{start_dt.date()}_{end_dt.date()}"

    # Try cache first
    cached = load_from_cache(cache_name)
    if cached is not None:
        return cached

    # Fetch from API
    print(f"  [API] Fetching equity history for {ticker} from {start_dt.date()} to {end_dt.date()}...")

    data = client.timeseries.get_range(
        dataset='XNAS.ITCH',
        symbols=[ticker],
        schema='ohlcv-1d',
        start=start_dt,
        end=end_dt + pd.Timedelta(days=1),
        stype_in='raw_symbol'
    )

    df = data.to_df(tz=CONFIG['timezone'])
    print(f"  [API] Fetched {len(df)} daily records")

    # Save to cache
    save_to_cache(df, cache_name)

    return df


def calculate_bollinger_bands(df, window=20, k=2.0):
    """Calculate Bollinger Bands and SMA on equity data"""
    df_bb = df.copy().sort_index()

    # Rolling stats on close
    roll = df_bb["close"].rolling(window=window, min_periods=window)
    df_bb["sma"] = roll.mean()
    df_bb["std"] = roll.std(ddof=0)

    # Bollinger Bands
    df_bb["bb_upper"] = df_bb["sma"] + k * df_bb["std"]
    df_bb["bb_lower"] = df_bb["sma"] - k * df_bb["std"]

    # Bollinger %B (position within bands)
    df_bb["bb_pctb"] = (df_bb["close"] - df_bb["bb_lower"]) / (df_bb["bb_upper"] - df_bb["bb_lower"])

    return df_bb


def check_technical_entry(ticker, entry_date, config):
    """
    Check if the technical entry conditions are met for a given date.

    Returns: (passes_filter: bool, details: dict)
    """
    if not config.get('technical_filter_enabled', False):
        return True, {'filter_enabled': False}

    # Need extra lookback for BB calculation
    lookback_days = config.get('bb_window', 20) + 40

    # Fetch equity history
    df_equity = fetch_equity_history(ticker, entry_date, lookback_days)

    # Calculate Bollinger Bands
    window = config.get('bb_window', 20)
    k = config.get('bb_std', 2.0)
    df_bb = calculate_bollinger_bands(df_equity, window=window, k=k)

    # Get the entry date row
    entry_dt = pd.Timestamp(entry_date).tz_localize(CONFIG['timezone']).normalize()

    # Find the closest date (in case entry_date is exact match or close)
    df_bb_dates = df_bb.index.normalize()

    # Try to find an exact or near match
    mask = df_bb_dates <= entry_dt
    if not mask.any():
        print(f"  [TECH FILTER] No data found for {entry_date}")
        return False, {'error': 'no_data'}

    # Get the most recent row on or before entry_date
    closest_idx = df_bb[mask].index[-1]
    row = df_bb.loc[closest_idx]

    close = row['close']
    sma = row['sma']
    bb_lower = row['bb_lower']

    # Check if we have valid BB data
    if pd.isna(sma) or pd.isna(bb_lower):
        print(f"  [TECH FILTER] Insufficient data for BB calculation on {entry_date}")
        return False, {'error': 'insufficient_data'}

    # Check entry conditions
    sma_entry = close <= sma
    bb_entry = close <= bb_lower

    details = {
        'date': closest_idx,
        'close': close,
        'sma': sma,
        'bb_lower': bb_lower,
        'sma_entry': sma_entry,
        'bb_entry': bb_entry,
    }

    # Determine if we pass the filter
    require_sma = config.get('require_sma_entry', True)
    require_bb = config.get('require_bb_entry', False)

    passes = False
    if require_bb:
        passes = bb_entry
    elif require_sma:
        passes = sma_entry
    else:
        passes = sma_entry or bb_entry  # Either condition

    return passes, details


print("Technical filter functions defined")

Technical filter functions defined


## 6. Exit Strategy Function

In [68]:
def backtest_exit_strategy(backtest_candidates, ticker, client, config):
    """
    Backtest exit strategy for wheel options with realistic simulation.
    
    Features:
    - Slippage on exit
    - Transaction costs
    - Fallback cascade when minute data unavailable
    - Tracks gross vs net P&L
    
    Exit conditions:
    1. Profit target: Exit when mid-price <= 50% of premium (early exit)
    2. Time limit: Force exit at exit_dte using fallback cascade
    """
    exits = []
    exit_dte = config.get('exit_dte', 21)
    profit_target_pct = config.get('profit_target_pct', 0.50)
    
    for idx, row in backtest_candidates.iterrows():
        symbol = row['symbol']
        
        # Normalize dates
        entry_date = pd.Timestamp(row['date']).tz_localize(None)
        expiration = pd.Timestamp(row['expiration']).tz_localize(None)
        date_exit = pd.Timestamp(row['date_exit']).tz_localize(None)
        strike = row['strike']
        
        # Entry details (already adjusted for slippage in process_entry_date)
        entry_premium = row['entry_premium']  # Slippage-adjusted entry price
        mid_premium = row['mid']              # Original mid price
        exit_target = entry_premium * profit_target_pct
        cost_basis = row['cost_basis']
        entry_costs = row['entry_costs']
        
        # Entry quote data (for logging)
        entry_bid = row.get('bid_px_00', 0)
        entry_ask = row.get('ask_px_00', 0)
        
        print(f"\nProcessing {symbol}...")
        print(f"  Entry: {entry_date.date()}, Mid: ${mid_premium:.2f}, Fill: ${entry_premium:.2f}")
        print(f"  Exit target: ${exit_target:.2f} ({profit_target_pct:.0%} of premium)")
        print(f"  Exit date ({exit_dte} DTE): {date_exit.date()}")
        
        exit_record = None
        
        try:
            # Fetch daily prices for monitoring profit target
            start_daily = entry_date + pd.Timedelta(days=1)
            end_daily = date_exit
            
            if start_daily > end_daily:
                print(f"  Warning: Invalid date range, skipping profit target check")
                df_daily = pd.DataFrame()
            else:
                df_daily = fetch_option_daily_ohlcv(symbol, start_daily, end_daily)
            
            # Check daily for profit target
            profit_target_hit = False
            
            for check_date, daily_row in df_daily.iterrows():
                daily_low = daily_row['low']
                daily_high = daily_row['high']
                
                # Check if exit target is within daily range
                if daily_low <= exit_target <= daily_high:
                    # Apply slippage to exit (buying to close)
                    # Use mid as approximation since we don't have intraday bid/ask
                    exit_mid = exit_target
                    # Estimate spread from daily range
                    estimated_spread = (daily_high - daily_low) * 0.5
                    exit_bid = exit_target - estimated_spread / 2
                    exit_ask = exit_target + estimated_spread / 2
                    exit_fill = apply_exit_slippage(exit_target, exit_bid, exit_ask, config)
                    
                    # Calculate costs
                    exit_costs = calculate_exit_costs(1, 'profit_target', config)
                    
                    # P&L calculations (per share)
                    gross_pnl = entry_premium - exit_fill
                    total_costs = entry_costs + exit_costs
                    net_pnl = gross_pnl - total_costs / 100  # Convert costs to per-share
                    
                    exit_record = {
                        'ticker': ticker,
                        'symbol': symbol,
                        'strike': strike,
                        'entry_date': entry_date,
                        'exit_date': check_date.tz_localize(None),
                        'expiration': expiration,
                        'cost_basis': cost_basis,
                        'entry_mid': mid_premium,
                        'entry_premium': entry_premium,
                        'exit_target': exit_target,
                        'exit_mid': exit_mid,
                        'exit_price': exit_fill,
                        'exit_reason': 'profit_target',
                        'exit_method': 'daily_range',
                        'days_held': (check_date.tz_localize(None) - entry_date).days,
                        'gross_pnl': gross_pnl,
                        'entry_costs': entry_costs,
                        'exit_costs': exit_costs,
                        'total_costs': total_costs,
                        'net_pnl': net_pnl,
                        'slippage_entry': mid_premium - entry_premium,
                        'slippage_exit': exit_fill - exit_target,
                    }
                    
                    print(f"  Profit target hit on {check_date.date()}")
                    print(f"    Fill: ${exit_fill:.2f}, Gross P&L: ${gross_pnl:.2f}, Net P&L: ${net_pnl:.2f}")
                    profit_target_hit = True
                    break
            
            # If profit target not hit, force exit at exit_dte using fallback
            if not profit_target_hit:
                # Get underlying price at exit date for ITM/OTM logic
                underlying_at_exit = fetch_underlying_price_for_date(ticker, date_exit.date())
                if underlying_at_exit is None:
                    underlying_at_exit = row.get('underlying_last', strike)  # Fallback to entry price
                
                # Use fallback cascade
                exit_price, exit_method = get_exit_price(
                    symbol, date_exit, expiration, strike, underlying_at_exit, config
                )
                
                if exit_price is not None:
                    # Apply slippage only for market exits (not expiry logic)
                    if exit_method in ['minute_data', 'daily_close']:
                        # Estimate spread as 5% of price for slippage calc
                        estimated_spread = exit_price * 0.05
                        exit_bid = exit_price - estimated_spread / 2
                        exit_ask = exit_price + estimated_spread / 2
                        exit_fill = apply_exit_slippage(exit_price, exit_bid, exit_ask, config)
                    else:
                        # No slippage for expiry (assigned or worthless)
                        exit_fill = exit_price
                    
                    # Calculate costs based on exit method
                    exit_costs = calculate_exit_costs(1, exit_method, config)
                    
                    # P&L calculations
                    gross_pnl = entry_premium - exit_fill
                    total_costs = entry_costs + exit_costs
                    net_pnl = gross_pnl - total_costs / 100
                    
                    exit_reason = f'time_limit_{exit_dte}dte' if exit_method in ['minute_data', 'daily_close'] else exit_method
                    
                    exit_record = {
                        'ticker': ticker,
                        'symbol': symbol,
                        'strike': strike,
                        'entry_date': entry_date,
                        'exit_date': date_exit,
                        'expiration': expiration,
                        'cost_basis': cost_basis,
                        'entry_mid': mid_premium,
                        'entry_premium': entry_premium,
                        'exit_target': exit_target,
                        'exit_mid': exit_price,
                        'exit_price': exit_fill,
                        'exit_reason': exit_reason,
                        'exit_method': exit_method,
                        'days_held': (date_exit - entry_date).days,
                        'gross_pnl': gross_pnl,
                        'entry_costs': entry_costs,
                        'exit_costs': exit_costs,
                        'total_costs': total_costs,
                        'net_pnl': net_pnl,
                        'slippage_entry': mid_premium - entry_premium,
                        'slippage_exit': exit_fill - exit_price if exit_method in ['minute_data', 'daily_close'] else 0,
                    }
                    
                    print(f"  Exit via {exit_method} on {date_exit.date()}")
                    print(f"    Fill: ${exit_fill:.2f}, Gross P&L: ${gross_pnl:.2f}, Net P&L: ${net_pnl:.2f}")
                else:
                    print(f"  ERROR: No exit price available - trade dropped")
            
            if exit_record:
                exits.append(exit_record)
                    
        except Exception as e:
            print(f"  Error: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    # Create results DataFrame
    exits_df = pd.DataFrame(exits)
    
    # Add ROC calculations
    if len(exits_df) > 0:
        exits_df['gross_pnl_pct'] = (exits_df['gross_pnl'] / exits_df['entry_premium']) * 100
        exits_df['net_pnl_pct'] = (exits_df['net_pnl'] / exits_df['entry_premium']) * 100
        exits_df['roc'] = (exits_df['net_pnl'] * 100 / exits_df['cost_basis']) * 100  # Multiply by 100 for per-contract
    
    return exits_df


print("Exit strategy function defined (v2 with costs, slippage, fallback)")

Exit strategy function defined (v2 with costs, slippage, fallback)


## 7. Process Entry Date Function

In [69]:
def process_entry_date(entry_date, ticker, positions_df, config):
    """
    Process a single entry date for a single ticker with realistic simulation.
    
    Features:
    - Liquidity filters
    - Slippage on entry
    - Transaction costs
    - Fixed cost basis calculation
    - Intermediate step caching for faster debugging
    
    Returns: (new_positions_df, exits_df, filtered_out_df)
    """
    print(f"\n{'='*60}")
    print(f"Processing {ticker} on {entry_date}")
    print('='*60)
    
    r = config['risk_free_rate']
    use_trading_days_filter = config.get('use_trading_days_for_dte_filter', True)
    use_trading_days_exit = config.get('use_trading_days_for_exit', True)
    exit_dte = config.get('exit_dte', 21)
    
    # ===========================================================================
    # STEP 1: Fetch options chain (uses API-level cache)
    # ===========================================================================
    df_opts = fetch_options_snapshot(ticker, entry_date)
    
    # ===========================================================================
    # STEP 2: Parse symbols
    # ===========================================================================
    df_parsed = load_step_cache('parsed_options', ticker, entry_date)
    if df_parsed is None:
        df_parsed = parse_option_symbols(df_opts)
        save_step_cache(df_parsed, 'parsed_options', ticker, entry_date)
    
    # ===========================================================================
    # STEP 3: Add DTE
    # ===========================================================================
    df_with_dte = load_step_cache('chain_with_dte', ticker, entry_date)
    if df_with_dte is None:
        df_with_dte = add_trading_dte(df_parsed, use_trading_days=use_trading_days_filter)
        save_step_cache(df_with_dte, 'chain_with_dte', ticker, entry_date)
    
    # ===========================================================================
    # STEP 4: Fetch underlying price (uses API-level cache)
    # ===========================================================================
    underlying_price = fetch_equity_price(ticker, entry_date)
    
    # ===========================================================================
    # STEP 5-7: Process chain snapshot with greeks
    # ===========================================================================
    chain_with_greeks = load_step_cache('chain_with_greeks', ticker, entry_date)
    if chain_with_greeks is None:
        # Keep only rows with quotes
        quotes = df_with_dte[df_with_dte["bid_px_00"].notna() & df_with_dte["ask_px_00"].notna()].copy()
        quotes["mid"] = (quotes["bid_px_00"] + quotes["ask_px_00"]) / 2
        
        # Collapse to one row per contract (latest quote)
        chain_snapshot = (
            quotes
            .sort_values("ts_event")
            .groupby(["symbol", "expiration", "strike", "call_put"])
            .tail(1)
            .copy()
        )
        chain_snapshot["underlying_last"] = underlying_price
        
        # Calculate IV and delta
        chain_snapshot["iv"] = chain_snapshot.apply(lambda row: compute_iv(row, r), axis=1)
        chain_snapshot["delta"] = chain_snapshot.apply(lambda row: compute_delta(row, r), axis=1)
        
        # Calculate exit dates
        chain_snapshot['date_exit'] = calculate_exit_dte_dates(
            chain_snapshot['expiration'], 
            exit_dte,
            use_trading_days=use_trading_days_exit
        )
        
        # Add entry date
        chain_snapshot['date'] = chain_snapshot['ts_event'].dt.date
        
        chain_with_greeks = chain_snapshot
        save_step_cache(chain_with_greeks, 'chain_with_greeks', ticker, entry_date)
    
    # ===========================================================================
    # STEP 8: Basic option filters (DTE, delta, type)
    # ===========================================================================
    candidates = load_step_cache('candidates', ticker, entry_date)
    if candidates is None:
        candidates = chain_with_greeks[
            (chain_with_greeks["call_put"] == config['option_type'])
            & chain_with_greeks["dte"].between(config['min_dte'], config['max_dte'])
            & chain_with_greeks["delta"].abs().between(config['min_delta'], config['max_delta'])
        ].copy()
        save_step_cache(candidates, 'candidates', ticker, entry_date)
    
    print(f"\nFound {len(candidates)} candidates passing basic filters (DTE, delta)")
    
    if len(candidates) == 0:
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    
    # 11. Apply liquidity filters
    filtered_out = []
    liquidity_passed = []
    
    for idx, row in candidates.iterrows():
        passes, reason = passes_liquidity_filter(row, config)
        if passes:
            liquidity_passed.append(idx)
        else:
            filtered_out.append({
                'symbol': row['symbol'],
                'strike': row['strike'],
                'dte': row['dte'],
                'reason': reason,
                'bid': row.get('bid_px_00', 0),
                'ask': row.get('ask_px_00', 0),
                'bid_size': row.get('bid_sz_00', 0),
                'ask_size': row.get('ask_sz_00', 0),
            })
    
    candidates = candidates.loc[liquidity_passed]
    filtered_out_df = pd.DataFrame(filtered_out)
    
    if config['liquidity']['enabled']:
        print(f"After liquidity filter: {len(candidates)} candidates")
        if len(filtered_out_df) > 0:
            print(f"  Filtered out: {len(filtered_out_df)} (reasons: {filtered_out_df['reason'].value_counts().to_dict()})")
    
    if len(candidates) == 0:
        return pd.DataFrame(), pd.DataFrame(), filtered_out_df
    
    # 12. Filter out same-day duplicates
    if len(positions_df) > 0:
        same_date_positions = positions_df[
            (positions_df['entry_date'] == entry_date) & 
            (positions_df['ticker'] == ticker)
        ]
        if len(same_date_positions) > 0:
            held_symbols = same_date_positions['symbol'].tolist()
            candidates = candidates[~candidates['symbol'].isin(held_symbols)]
            print(f"After removing same-day duplicates: {len(candidates)} candidates")
    
    if len(candidates) == 0:
        return pd.DataFrame(), pd.DataFrame(), filtered_out_df
    
    # 13. Create backtest candidates with slippage and costs
    backtest_candidates = candidates.copy()
    
    # Apply entry slippage (selling puts)
    backtest_candidates['entry_premium'] = backtest_candidates.apply(
        lambda row: apply_entry_slippage(
            row['mid'], 
            row['bid_px_00'], 
            row['ask_px_00'], 
            config
        ), 
        axis=1
    )
    
    # Calculate cost basis (FIXED)
    backtest_candidates['cost_basis'] = backtest_candidates['strike'].apply(
        lambda s: calculate_cost_basis(s, config)
    )
    
    # Calculate entry costs
    backtest_candidates['entry_costs'] = calculate_entry_costs(1, config)
    
    # Calculate profit target based on slippage-adjusted premium
    backtest_candidates['exit_50_perc'] = config['profit_target_pct'] * backtest_candidates['entry_premium']
    
    # Track slippage
    backtest_candidates['entry_slippage'] = backtest_candidates['mid'] - backtest_candidates['entry_premium']
    
    # Select columns for output
    backtest_cols = [
        'symbol', 'date_exit', 'cost_basis', 'mid', 'entry_premium', 'entry_costs',
        'exit_50_perc', 'entry_slippage', 'date', 'dte', 'expiration', 'strike',
        'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00', 'underlying_last'
    ]
    backtest_candidates = backtest_candidates[backtest_cols]
    
    print(f"\nBacktest candidates:")
    display_cols = ['symbol', 'strike', 'dte', 'mid', 'entry_premium', 'entry_slippage']
    print(backtest_candidates[display_cols].to_string())
    
    # 14. Run exit strategy
    exits_df = backtest_exit_strategy(backtest_candidates, ticker, client, config)
    
    # 15. Create new positions for tracking
    new_positions = backtest_candidates.copy()
    new_positions['entry_date'] = entry_date
    new_positions['ticker'] = ticker
    
    return new_positions, exits_df, filtered_out_df


print("Process entry date function defined (v2 with liquidity, slippage, costs)")

Process entry date function defined (v2 with liquidity, slippage, costs)


## 8. Main Backtest Loop

In [ ]:
# Initialize tracking DataFrames
positions_df = pd.DataFrame()      # All positions entered
all_exits_df = pd.DataFrame()      # All completed trades
all_filtered_df = pd.DataFrame()   # Options filtered out by liquidity
skipped_entries = []               # Ticker/date combos that failed technical filter

# Process each ticker and entry date combination
for ticker in CONFIG['tickers']:
    print(f"\n{'#'*70}")
    print(f"# Processing ticker: {ticker}")
    print('#'*70)

    for entry_date in CONFIG['entry_dates']:

        # Check technical filter first (per ticker)
        if CONFIG.get('technical_filter_enabled', False):
            passes_filter, tech_details = check_technical_entry(
                ticker, entry_date, CONFIG
            )

            if not passes_filter:
                print(f"\n[SKIPPED] {ticker} on {entry_date} - Failed technical filter")
                if 'close' in tech_details:
                    print(f"  Close: ${tech_details['close']:.2f}, SMA: ${tech_details['sma']:.2f}")
                skipped_entries.append({'ticker': ticker, 'date': entry_date, 'reason': 'technical_filter', **tech_details})
                continue
            else:
                print(f"\n[PASSED] {ticker} on {entry_date} - Technical filter passed")
                if 'close' in tech_details:
                    print(f"  Close: ${tech_details['close']:.2f}, SMA: ${tech_details['sma']:.2f}")

        # Process this ticker/date combination
        new_positions, exits, filtered = process_entry_date(
            entry_date=entry_date,
            ticker=ticker,
            positions_df=positions_df,
            config=CONFIG
        )

        # Add new positions
        if len(new_positions) > 0:
            positions_df = pd.concat([positions_df, new_positions], ignore_index=True)

        # Accumulate exits
        if len(exits) > 0:
            all_exits_df = pd.concat([all_exits_df, exits], ignore_index=True)

        # Track filtered out options
        if len(filtered) > 0:
            filtered['ticker'] = ticker
            filtered['entry_date'] = entry_date
            all_filtered_df = pd.concat([all_filtered_df, filtered], ignore_index=True)

print(f"\n{'='*70}")
print("BACKTEST COMPLETE")
print('='*70)
print(f"Tickers processed: {CONFIG['tickers']}")
total_combinations = len(CONFIG['tickers']) * len(CONFIG['entry_dates'])
print(f"Total ticker/date combinations: {total_combinations}")

if CONFIG.get('technical_filter_enabled', False) and len(skipped_entries) > 0:
    print(f"Skipped by technical filter: {len(skipped_entries)}")

print(f"\nTotal positions entered: {len(positions_df)}")
print(f"Total exits: {len(all_exits_df)}")

if len(all_filtered_df) > 0:
    print(f"Options filtered by liquidity: {len(all_filtered_df)}")

## 9. Results - Aggregate

In [ ]:
if len(all_exits_df) > 0:
    print("\n" + "="*70)
    print("AGGREGATE RESULTS (All Tickers)")
    print("="*70)
    
    # Exit method breakdown
    print("\nExit Methods:")
    print(all_exits_df['exit_method'].value_counts().to_string())
    
    print("\nExit Reasons:")
    print(all_exits_df['exit_reason'].value_counts().to_string())
    
    # P&L Summary
    print("\n" + "-"*40)
    print("P&L SUMMARY (per share values)")
    print("-"*40)
    
    total_gross = all_exits_df['gross_pnl'].sum()
    total_net = all_exits_df['net_pnl'].sum()
    total_costs = all_exits_df['total_costs'].sum()
    total_entry_slippage = all_exits_df['slippage_entry'].sum()
    total_exit_slippage = all_exits_df['slippage_exit'].sum()
    
    print(f"\nGross P&L:     ${total_gross:>10.2f}")
    print(f"Total Costs:   ${total_costs:>10.2f}")
    print(f"Net P&L:       ${total_net:>10.2f}")
    print(f"\nEntry Slippage: ${total_entry_slippage:>9.2f}")
    print(f"Exit Slippage:  ${total_exit_slippage:>9.2f}")
    
    # Per-contract values (x100)
    print("\n" + "-"*40)
    print("P&L SUMMARY (per contract = x100)")
    print("-"*40)
    print(f"\nGross P&L:     ${total_gross * 100:>10.2f}")
    print(f"Total Costs:   ${total_costs:>10.2f}")
    print(f"Net P&L:       ${total_net * 100:>10.2f}")
    
    # Performance metrics
    print("\n" + "-"*40)
    print("PERFORMANCE METRICS")
    print("-"*40)
    print(f"\nTrades: {len(all_exits_df)}")
    print(f"Win Rate: {(all_exits_df['net_pnl'] > 0).mean() * 100:.1f}%")
    print(f"Avg Gross P&L: ${all_exits_df['gross_pnl'].mean():.2f}")
    print(f"Avg Net P&L:   ${all_exits_df['net_pnl'].mean():.2f}")
    print(f"Avg ROC:       {all_exits_df['roc'].mean():.2f}%")
    print(f"Avg Days Held: {all_exits_df['days_held'].mean():.1f}")
    
    # Trade log
    print("\n" + "-"*40)
    print("TRADE LOG")
    print("-"*40)
    display_cols = ['ticker', 'symbol', 'entry_date', 'exit_date', 
                   'entry_premium', 'exit_price', 'gross_pnl', 'net_pnl', 
                   'total_costs', 'roc', 'exit_method']
    print(all_exits_df[display_cols].to_string())
else:
    print("No exits recorded")

## 10. Results - By Ticker

In [ ]:
if len(all_exits_df) > 0:
    print("\n" + "="*70)
    print("RESULTS BY TICKER")
    print("="*70)
    
    for ticker in CONFIG['tickers']:
        ticker_exits = all_exits_df[all_exits_df['ticker'] == ticker]
        
        if len(ticker_exits) == 0:
            print(f"\n{ticker}: No exits")
            continue
            
        print(f"\n{'-'*50}")
        print(f"{ticker}")
        print(f"{'-'*50}")
        
        print(f"  Trades: {len(ticker_exits)}")
        print(f"  Win Rate: {(ticker_exits['net_pnl'] > 0).mean() * 100:.1f}%")
        print(f"\n  Gross P&L: ${ticker_exits['gross_pnl'].sum():.2f} (per share)")
        print(f"  Net P&L:   ${ticker_exits['net_pnl'].sum():.2f} (per share)")
        print(f"  Costs:     ${ticker_exits['total_costs'].sum():.2f}")
        print(f"\n  Avg Gross: ${ticker_exits['gross_pnl'].mean():.2f}")
        print(f"  Avg Net:   ${ticker_exits['net_pnl'].mean():.2f}")
        print(f"  Avg ROC:   {ticker_exits['roc'].mean():.2f}%")
        print(f"  Avg Days:  {ticker_exits['days_held'].mean():.1f}")
        
        print(f"\n  Exit methods:")
        for method, count in ticker_exits['exit_method'].value_counts().items():
            print(f"    {method}: {count}")
else:
    print("No exits recorded")

## 11. Ticker Comparison Summary

In [ ]:
if len(all_exits_df) > 0:
    print("\n" + "="*70)
    print("TICKER COMPARISON")
    print("="*70)
    
    comparison = all_exits_df.groupby('ticker').agg({
        'gross_pnl': ['count', 'sum', 'mean'],
        'net_pnl': ['sum', 'mean'],
        'total_costs': 'sum',
        'roc': 'mean',
        'days_held': 'mean',
    }).round(2)
    
    comparison.columns = ['Trades', 'Gross P&L', 'Avg Gross', 
                          'Net P&L', 'Avg Net', 'Total Costs',
                          'Avg ROC %', 'Avg Days']
    
    # Add win rate
    win_rates = all_exits_df.groupby('ticker').apply(
        lambda x: (x['net_pnl'] > 0).mean() * 100,
        include_groups=False
    ).round(1)
    comparison['Win %'] = win_rates
    
    print(comparison.to_string())
    
    # Cost impact summary
    print("\n" + "-"*40)
    print("COST & SLIPPAGE IMPACT")
    print("-"*40)
    
    for ticker in CONFIG['tickers']:
        t = all_exits_df[all_exits_df['ticker'] == ticker]
        if len(t) == 0:
            continue
        
        gross = t['gross_pnl'].sum()
        net = t['net_pnl'].sum()
        costs = t['total_costs'].sum()
        slip_entry = t['slippage_entry'].sum()
        slip_exit = t['slippage_exit'].sum()
        
        print(f"\n{ticker}:")
        print(f"  Gross → Net: ${gross:.2f} → ${net:.2f}")
        print(f"  Costs: ${costs:.2f} ({costs/gross*100 if gross else 0:.1f}% of gross)")
        print(f"  Entry slippage: ${slip_entry:.2f}")
        print(f"  Exit slippage: ${slip_exit:.2f}")
else:
    print("No exits recorded")

## 12. Validation & Filtered Options

Check trades and see which options were filtered by liquidity.

In [ ]:
# Show options filtered out by liquidity
if len(all_filtered_df) > 0:
    print("="*60)
    print("OPTIONS FILTERED BY LIQUIDITY")
    print("="*60)
    print(f"\nTotal filtered: {len(all_filtered_df)}")
    print("\nBy reason:")
    for reason, count in all_filtered_df['reason'].value_counts().items():
        print(f"  {reason}: {count}")
    
    print("\nSample filtered options:")
    print(all_filtered_df.head(10).to_string())
else:
    print("No options filtered by liquidity")

# Validation: Check specific symbols
print("\n" + "="*60)
print("VALIDATION")
print("="*60)

expected_tsla = ['TSLA  230721P00200000', 'TSLA  230721P00205000']

if len(all_exits_df) > 0:
    print("\nTSLA validation (expected symbols):")
    for sym in expected_tsla:
        match = all_exits_df[all_exits_df['symbol'] == sym]
        if len(match) > 0:
            row = match.iloc[0]
            print(f"\n  {sym}:")
            print(f"    Entry: ${row['entry_premium']:.2f} (mid: ${row['entry_mid']:.2f})")
            print(f"    Exit:  ${row['exit_price']:.2f} via {row['exit_method']}")
            print(f"    Gross P&L: ${row['gross_pnl']:.2f}")
            print(f"    Net P&L:   ${row['net_pnl']:.2f}")
            print(f"    ROC: {row['roc']:.2f}%")
        else:
            print(f"\n  {sym}: NOT FOUND")
else:
    print("\nNo trades to validate")